# <center><u> Face Detection using MTCNN </u></center>

<center><img src="https://play-lh.googleusercontent.com/_TPNZQF4zJ8XWvPsNxVAQk4ogODPhcwojnD1YFzIaLT675mW_DtT-5pgav2tO0WHIbs=w240-h480-rw"></center>

## why Face detection and why only MTCNN?

Facial recognition task requires the creation of face embeddings for the faces that we want to recognize. All models that are capable of generating face embeddings will generate them properly if and only if the image that contains only the face to it is passed. So, most of the face recogntion pipe lines will have the face detection and extraction as the preliminary task. There are various models available in the community that are capable of detecting the faces in the image. MTCNN(Multi Task Cascaded Nueral Network) is one of the start of the art techniques for face detection. Though the execution time is a bit high when compared to other models, the performance of the model is best and is capable of detecting faces well in different light conditions. 

### Installing MTCNN

<center><img src="https://miro.medium.com/v2/resize:fit:490/1*mH7AABb6oha9g6v9jB4gjw.png"> </center>

In [ ]:
! pip install mtcnn

### Installing few other required packages.

1. **shutil** - Used for creating, deleting and moving files and directores.
2. **tensorflow** - Framework for building deep learning networks.
3. **mtcnn** - contains the architectue of the model and helps in inferencing MTCNN.
4. **cv2(opencv)** - Used for performing operations on images like colour format conversion, resizing and many more.
5. **tqdm** - Instantly make loops show a smart progress meter.
6. **matplotlib** - Used for plotting. Sometimes can also be used for displaying images.
7. **glob** - Used for listing files in a directory recursivley using wildcards.
8. **random** - Used for random number generation with different distributions.
9. **os** - Used for accessing files and moving across file system.

In [ ]:
import shutil
import tensorflow as tf
from mtcnn.mtcnn import MTCNN
import cv2
import tqdm
import matplotlib.pyplot as plt
import glob
import random
import os

## Lets have a look at the folder structure of our data.

In [ ]:
! tree -d "/kaggle/input/indian-actor-images-dataset"

We have 135 directories with 135 unique indian actor names. Each directory will have 50 unique images of that corresponding actor.

### Displaying one of the image in the dataset

In [ ]:
img = cv2.imread('/kaggle/input/indian-actor-images-dataset/Bollywood Actor Images/Bollywood Actor Images/deepika_padukone/03fcc8cb3a.jpg')
plt.tick_params(left=False, bottom=False, labelleft=False, labelbottom=False)
plt.imshow(img)
plt.show()

## What is the color format of above image and why is it differnet to our original image?

We have read the above image using opencv. The opencv reads the image in BGR(Blue Green Red) colour format by default. Our original image is in the RGB(Red Green Blue) colour format. To look at the original image, we can change the colour format of the image to RGB from BGR using ```cvtColor()``` of Opencv.  

In [ ]:
plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
plt.tick_params(left=False, bottom=False, labelleft=False, labelbottom=False)
plt.show()

### Generating a grid of random images from the dataset

In [ ]:
def grid_creation(num_rows, num_columns, image_paths):
    """
    Params:
     - num_rows : number of rows to be present in the grid of images.
     - num_columns :  number of columns to be present in the grid of images.
     - image_path : list with paths to all the images in the dataset.
    Functionality:
     Generates a grid of images according to the input dimensions from the randomly selected images
     availbale in the dataset.
    """
    fig,axes = plt.subplots(num_rows, num_columns, figsize=(18,11), sharex=True, sharey=True)
    for i,axes in enumerate(axes.flat):
        random_image_path = random.choice(image_paths)
        random_image = cv2.imread(random_image_path)
        random_image = cv2.resize(random_image, (250,250))
        random_image = cv2.cvtColor(random_image, cv2.COLOR_BGR2RGB)
        axes.set_xlabel(random_image_path.split('/')[-2])
        axes.set_yticks([])
        axes.imshow(random_image)

In [ ]:
directory_path = '/kaggle/input/indian-actor-images-dataset/Bollywood Actor Images/Bollywood Actor Images'
image_paths = glob.glob(directory_path+'/*/*')

In [ ]:
grid_creation(4, 8, image_paths)

## Lets understand How to leverage MTCNN for face detection and its output format.

1. The results from MTCNN will be a list of dictonaries where each dictionary contains the information about each detected face in the image. So, the length of the result from MTCNN represents the total number of faces detected in the image.

2. Each dictionary will have 3 keys. They are
   ```
   a. box : list of 4 numbers and each number has its own significance. The first and the second               number represents the X,Y co-ordinates of the top left corner of the bounding box that             covers the face. The third and the fourth numbers represent the height and widht of                 that bounding box.
   Format -> 'box':[90,23,73,96]
   
   b. confidence : This value lies between a range of 0 and 1 representing the authenticity of the                    detected face.
   Format -> 'confidence':0.9988888888
   
   c. keypoints : This is again a dictionary with 5 keys in it. This dictinary will contians the X,                   Y cordinates of left_eye, right_eye, nose and mouth.
    Format -> 'keypoints':{'left_eye': (104, 63),'right_eye': (137, 58),'nose': (119,80),
    'mouth_left':(110,96),'mouth_right': (144, 91)}}
  ```

### Overlaying the results of MTCNN on one of the image.

In [ ]:
random_image_path = random.choice(image_paths)
random_image = cv2.cvtColor(cv2.imread(random_image_path), cv2.COLOR_BGR2RGB)
detector = MTCNN()
result = detector.detect_faces(random_image)
if len(result)>1:
    print('$'*50)
    print("ReExute the cell once again.")
    print('$'*50)
else:
    x1, y1, width, height = result[0]['box'][0], result[0]['box'][1], result[0]['box'][2], result[0]['box'][3],
    x2, y2 = x1 + width, y1 + height
    # Displaying the bounding box over the face of the actor
    random_image=cv2.rectangle(random_image, (x1,y1), (x2,y2),(255,0,0), 4)
    
    # Displaying the confidence value for the detected bounding box.
    random_image=cv2.putText(random_image,f"confidence:{round(result[0]['confidence'],2)}",
                             (x1-50,y2+10),cv2.FONT_HERSHEY_COMPLEX_SMALL,1,(0,255,255),1)
    
    # Displaying the detected keypoings in the face
    for key in result[0]['keypoints']:
        random_image=cv2.circle(random_image,result[0]['keypoints'][key],radius=0,color=(255,0,0),thickness=5)
    plt.xlabel(random_image_path.split('/')[-2])
    plt.tick_params(left=False, bottom=False, labelleft=False, labelbottom=False)
    plt.imshow(random_image)

In [ ]:
def return_face(detector,img_path):
    """
    Params:
     - detector : instance of MTCNN class.
     - img_path : Path to the image
    Functionality:
     Functon detects the face in the image using MTCNN and extracts the pixels of the image and'
     returns a new image that has only detected face in it.
    """
    img=cv2.imread(img_path)
    img=cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    face=detector.detect_faces(img)
    if len(face)>1 or len(face)<1:
        print(img_path)
        return []
    face_coords=face[0]['box']
    x1, y1, width, height = face_coords[0],face_coords[1],face_coords[2],face_coords[3],
    x2, y2 = x1 + width, y1 + height
    extracted_face=img[y1:y2,x1:x2]
    return extracted_face

In [ ]:
def move_images(actor_name,base_dir,destination_dir,images):
    """
    Params:
     - actor_name : Name of the actor. This is used to create a directory.
     - base_dir : path to the directory that contains the original base images.
     - destination_dir : Destination directory path that should contains the processed images
     - images : list of images present in the directory of corresponding actor.
    Functionality
     Function works at an actor level. The function takes each actor images once, processes them and
     saves the processed images in a new directory. The base directory structure is maintained in the
     destination as well. 
    """
    excluded_image_paths=[]
    actor_dir_path=destination_dir+actor_name
    os.mkdir(actor_dir_path)
    detector=MTCNN()
    for image in images:
        face=return_face(detector,base_dir+actor_name+'/'+image)
        if len(face)==0:
            excluded_image_paths.append(base_dir+actor_name+'/'+image)
            continue
        face=cv2.cvtColor(face, cv2.COLOR_RGB2BGR)
        cv2.imwrite(actor_dir_path+'/'+image,face)
    return excluded_image_paths

## Extracting the face images from all the images in the dataset.

In [ ]:
base_dir='/kaggle/input/indian-actor-images-dataset/Bollywood Actor Images/Bollywood Actor Images/'
destination_dir='/kaggle/working/bollywood_actors_dataset/'
os.mkdir(destination_dir)
excluded_images=[]
dir_count=1
for actor_dir in tqdm.tqdm(os.listdir(base_dir)):
    print(f"Directory Count : {dir_count}")
    actor_images=os.listdir(base_dir+actor_dir)
    excluded_image_paths=move_images(actor_dir,base_dir,destination_dir,actor_images)
    excluded_images.extend(excluded_image_paths)
    dir_count+=1

While detecing the faces in the images, we have left few images from processing. Those images include ones in which MTCNN did not detect atleast one face and the images in which MTCNN has detected more than one face in it. The reasons for excluding images that has more than one face in it was, there might be a chance where the image contains the face of some other actor. If we save both the faces in the one directory, we might end up having the wrong faces in each directory. Instead, we will just save the paths of the images that we have excluded. If the excluded images count is more, we can manually crop those images and if the count is less, we can exclude them from the dataset. For the images in which the MTCNN did not detect any face in it, we can go ahead an try some other face detectoin technique that would reduce this count a bit further. 

In [ ]:
print(f"Total number of Excluded Images: {len(excluded_images)}")
percentage_dropped=round((len(excluded_images)/len(os.listdir(base_dir)*50))*100,2)
print(f"Percentage of Images dropped: {percentage_dropped}%")

In [ ]:
! tree -d "/kaggle/working/bollywood_actors_dataset"

### Generating grid of images from the processed images

In [ ]:
directory_path = '/kaggle/working/bollywood_actors_dataset'
image_paths = glob.glob(directory_path+'/*/*')
grid_creation(4, 4, image_paths)

### Downloading the created dataset.

Downloading the generated images from the output section of kaggle notebook using the UI download option is not an smart idea to export our dataset. This is so because, we can only download one image at once via the UI download option. We have greater than 1500 images and we need to click the download button for 1500 times. Instead, we can zip our entire processed dataset and can download it with just one click. 

In [ ]:
!zip -r file.zip /kaggle/working/bollywood_actors_dataset

In [ ]:
# you can also download the dataset using the below link
from IPython.display import FileLink
FileLink(r'file.zip')

<center style="color:red;font-size:20px">Upvote the notebook, if this has added any value to your learnings.</center>

In [ ]:
nan